In [1]:
import sys
sys.path.insert(0, 'C:\\Users\\Paarth Tandon\\Desktop\\repos\\personal-music-gen')

In [2]:
from audiocraft.models import MusicGen
from personal_musicgen.data.datasets import AudioDataset
from personal_musicgen.model_utils import *

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.cuda.amp import GradScaler
from torch.utils.data import DataLoader, Subset
import torchaudio

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
c:\Users\Paarth Tandon\Desktop\repos\personal-music-gen\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = AudioDataset('../data/chunks_no_voice/')
model = MusicGen.get_pretrained('small')
model.lm = model.lm.to(torch.float32)

c:\users\paarth tandon\desktop\repos\personal-music-gen\libs\audiocraft\audiocraft\models\musicgen.py:116: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-small
  warnings.warn(
c:\Users\Paarth Tandon\Desktop\repos\personal-music-gen\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [4]:
model.device

device(type='cuda', index=0)

In [5]:
dataloader = DataLoader(Subset(data, list(range(5))), batch_size=1)
acc_steps = 4
optimizer = AdamW(
    model.lm.parameters(),
    lr=1e-5,
    betas=(0.9, 0.95),
    weight_decay=0.1,
)
scaler = torch.cuda.amp.GradScaler()

In [6]:
train_step(model, optimizer, scaler, dataloader, acc_steps)

100%|██████████| 5/5 [00:09<00:00,  1.99s/it]


{'train_loss': 3.4339113235473633}

In [7]:
eval_step(model, dataloader)

100%|██████████| 5/5 [00:07<00:00,  1.48s/it]


{'eval_loss': 3.383491277694702}

In [8]:
# prompt = 'personal, country'
# attributes, prompt_tokens = model._prepare_tokens_and_attributes([prompt], None)

In [9]:
# import tqdm

# duration = 30

# model.generation_params = {
#     'max_gen_len': int(duration * model.frame_rate),
#     'use_sampling': True,
#     'temp': 1.0,
#     'top_k': 250,
#     'top_p': 0.0,
#     'cfg_coef': 3,
#     'two_step_cfg': 0,
# }

# gen_chunks = []
# for _ in tqdm.tqdm(range(1)):
#     with model.autocast:
#         gen_tokens = model.lm.generate(prompt_tokens, attributes, callback=None, **model.generation_params)
#         gen_chunks.append(gen_tokens[..., prompt_tokens.shape[-1] if prompt_tokens is not None else 0:])
#         prompt_tokens = gen_tokens[..., -gen_tokens.shape[-1] // 2:]
# gen_tokens = torch.cat(gen_chunks, -1)

In [10]:
# assert gen_tokens.dim() == 3
# print("gen_tokens information")
# print("Shape:", gen_tokens.shape)
# print("Dtype:", gen_tokens.dtype)
# print("Contents:", gen_tokens)
# with torch.no_grad():
#     gen_audio = model.compression_model.decode(gen_tokens, None)
# print("gen_audio information")
# print("Shape:", gen_audio.shape)
# print("Dtype:", gen_audio.dtype)
# print("Contents:", gen_audio)
# gen_audio = gen_audio.cpu()

In [15]:
model.frame_rate

50

In [13]:
prompt = 'personal, hip-hop'
gen_audio = generate(model, prompt, 1)

100%|██████████| 1/1 [00:50<00:00, 50.19s/it]


In [14]:
from audiocraft.utils.notebook import display_audio

display_audio(gen_audio, 32000)